In [3]:
import torch
import dgl

In [4]:
dataset = dgl.data.GINDataset("PROTEINS", self_loop=True)

Extracting file to /Users/marc/.dgl/GINDataset


In [6]:
dataset.graph_path

'/Users/marc/.dgl/PROTEINS_0c2c49a1/gin_PROTEINS_c0c5f972.bin'

In [8]:
dataset.graphs

[Graph(num_nodes=42, num_edges=204,
       ndata_schemes={'label': Scheme(shape=(), dtype=torch.int64), 'attr': Scheme(shape=(3,), dtype=torch.float32)}
       edata_schemes={}),
 Graph(num_nodes=27, num_edges=119,
       ndata_schemes={'label': Scheme(shape=(), dtype=torch.int64), 'attr': Scheme(shape=(3,), dtype=torch.float32)}
       edata_schemes={}),
 Graph(num_nodes=10, num_edges=44,
       ndata_schemes={'label': Scheme(shape=(), dtype=torch.int64), 'attr': Scheme(shape=(3,), dtype=torch.float32)}
       edata_schemes={}),
 Graph(num_nodes=24, num_edges=116,
       ndata_schemes={'label': Scheme(shape=(), dtype=torch.int64), 'attr': Scheme(shape=(3,), dtype=torch.float32)}
       edata_schemes={}),
 Graph(num_nodes=11, num_edges=53,
       ndata_schemes={'label': Scheme(shape=(), dtype=torch.int64), 'attr': Scheme(shape=(3,), dtype=torch.float32)}
       edata_schemes={}),
 Graph(num_nodes=336, num_edges=1968,
       ndata_schemes={'label': Scheme(shape=(), dtype=torch.int64), '